In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url = 'https://www.gpost.ge/?site-lang=en&site-path=company/servicemap/&mode=list'
response = requests.get(url)
soup = BeautifulSoup(response.content,'html.parser')
table = soup.find_all('div',class_='spb-04')
# my_table.append(table)

In [ ]:
del table[56:]

In [ ]:
for i in range(56):
    table[i]=table[i].text

In [ ]:
table

In [ ]:
PostCodes = []
Address = []

In [ ]:
for i in range(1,56,4):
    PostCodes.append(table[i][-4:])
    Address.append(table[i+1][9:])

In [ ]:
PostCodes

In [ ]:
Address

In [ ]:
my_data = {'Address':[],'PostCode':[],'Latitude':[],'Longitude':[]}

In [ ]:
my_data = pd.DataFrame(my_data)

In [ ]:
my_data['Address'] = Address
my_data['PostCode'] = PostCodes

In [ ]:
my_data['Address'][3]='6 Grigol Robakidze Ave, Tbilisi, Georgia'
my_data['Address'][12]='Tbilisi, Gldani'

In [ ]:
my_data

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException

In [ ]:
path = 'C:\\GeckoDriver\\geckodriver.exe'

driver = webdriver.Firefox(executable_path = path)

driver.set_page_load_timeout(16)

driver.get('https://www.gps-coordinates.net/gps-coordinates-converter')
lat = []
long = []
for i in my_data['Address']: 
    driver.find_element_by_xpath('//*[@id="address"]').send_keys(i)
    driver.find_element_by_css_selector('form.form-horizontal:nth-child(1)>div:nth-child(2)>div:nth-child(1)>button:nth-child(1)').click()
    
    time.sleep(6)
    
    latitude = driver.find_element_by_id('latitude').get_attribute('value')
    lat.append(latitude)
    longitude = driver.find_element_by_id('longitude').get_attribute('value')
    long.append(longitude)
    driver.find_element_by_css_selector('button.ap-input-icon:nth-child(4)').click()
    time.sleep(2)

In [ ]:
my_data['Latitude'] = lat
my_data['Longitude'] = long

In [1]:
my_data

NameError: name 'my_data' is not defined

In [ ]:
my_data.to_csv('C:\\Users\\admin\\Desktop\\Job\\Project_1\\Grigor\\Tbilisi_postcode.csv', encoding='utf-8', index=True)

In [48]:
my_data = pd.read_csv('C:\\Users\\admin\\Desktop\\Job\\Project_1\\Grigor\\Tbilisi_postcode.csv')

In [4]:
CLIENT_ID = 'BNWKW22EZCDC2PBW13BYIP0H2H3VVCBR1LKCXMPF4METKIXP'
CLIENT_SECRET = 'YPPP00QC5GBB0LADRW1SXGB22OAVRWOZDIVSJ3OJGQFRT35C' # your Foursquare Secret
VERSION = '20190217'

LIMIT = 500
radius = 3500

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=3500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Address', 
                  'Address Latitude', 
                  'Address Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
tbilisi_venues = getNearbyVenues(names=my_data['Address'],
                                   latitudes=my_data['Latitude'],
                                   longitudes=my_data['Longitude']
                                  )

Tbilisi, D.Guramishvili ave.№39b
Tbilisi, Ketevan Tsamebuli ave.№84/86
Tbilisi, Station Square №2
6 Grigol Robakidze Ave, Tbilisi, Georgia
Tbilisi, A.Tsereteli ave.№95
Tbilisi, Lilo Settlement, II kvart, building №9
Tbilisi, Moscow avenue №44
Tbilisi, Kakheti Highway №34b
Tbilisi, V.Gorgasali str.№4
Tbilisi, I.Chavchavadze av.№21
Tbilisi, Pekini ave.№39
Tbilisi, Vaja-Pshavela ave.№69
Tbilisi, Gldani
Tbilisi,Ts. Dadiani str.№315


In [7]:
tbil_onehot = pd.get_dummies(tbilisi_venues[['Venue Category']], prefix="", prefix_sep="")

tbil_onehot['Address'] = tbilisi_venues['Address'] 

fixed_columns = [tbil_onehot.columns[-1]] + list(tbil_onehot.columns[:-1])
tbil_onehot = tbil_onehot[fixed_columns]

In [8]:
tbil_onehot.head(10)

,Address,American Restaurant,Arepa Restaurant,Art Gallery,Asian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bakery,...,Theme Restaurant,Train Station,Turkish Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Volleyball Court,Warehouse Store,Waterfall,Wine Bar,Wine Shop
0,"Tbilisi, D.Guramishvili ave.№39b",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Tbilisi, D.Guramishvili ave.№39b",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Tbilisi, D.Guramishvili ave.№39b",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Tbilisi, D.Guramishvili ave.№39b",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Tbilisi, D.Guramishvili ave.№39b",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Tbilisi, D.Guramishvili ave.№39b",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Tbilisi, D.Guramishvili ave.№39b",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Tbilisi, D.Guramishvili ave.№39b",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Tbilisi, D.Guramishvili ave.№39b",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Tbilisi, D.Guramishvili ave.№39b",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
tbil_onehot.to_csv('C:\\Users\\admin\\Desktop\\Job\\Project_1\\Grigor\\Tbilisi_caffees.csv', encoding='utf-8', index=True)

In [ ]:
cafe_data = tbil_onehot.T

In [ ]:
cafe_data.to_csv('C:\\Users\\admin\\Desktop\\Job\\Project_1\\Grigor\\Cafee_data.csv', encoding='utf-8', index=True)

In [9]:
tbil_grouped = tbil_onehot.groupby('Address').mean().reset_index()
tbil_grouped.head(7)

,Address,American Restaurant,Arepa Restaurant,Art Gallery,Asian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bakery,...,Theme Restaurant,Train Station,Turkish Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Volleyball Court,Warehouse Store,Waterfall,Wine Bar,Wine Shop
0,"6 Grigol Robakidze Ave, Tbilisi, Georgia",0.0,0.000000,0.0,0.000000,0.011236,0.000000,0.011236,0.0,0.033708,...,0.00,0.000000,0.000000,0.011236,0.00,0.000000,0.011236,0.00,0.00,0.00
1,"Tbilisi, A.Tsereteli ave.№95",0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.070000,...,0.00,0.000000,0.000000,0.010000,0.01,0.000000,0.000000,0.00,0.00,0.01
2,"Tbilisi, D.Guramishvili ave.№39b",0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.022727,...,0.00,0.011364,0.011364,0.000000,0.00,0.011364,0.000000,0.00,0.00,0.00
3,"Tbilisi, Gldani",0.0,0.000000,0.0,0.000000,0.000000,0.015385,0.000000,0.0,0.030769,...,0.00,0.000000,0.015385,0.000000,0.00,0.015385,0.000000,0.00,0.00,0.00
4,"Tbilisi, I.Chavchavadze av.№21",0.0,0.000000,0.0,0.010000,0.000000,0.000000,0.000000,0.0,0.030000,...,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.00,0.03,0.02
5,"Tbilisi, Kakheti Highway №34b",0.0,0.013333,0.0,0.013333,0.000000,0.000000,0.000000,0.0,0.026667,...,0.00,0.000000,0.013333,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00
6,"Tbilisi, Ketevan Tsamebuli ave.№84/86",0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.020000,...,0.01,0.000000,0.010000,0.000000,0.00,0.000000,0.000000,0.02,0.06,0.02


In [10]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [11]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Address']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

addresses_venues_sorted = pd.DataFrame(columns=columns)
addresses_venues_sorted['Address'] = tbil_grouped['Address']

for ind in np.arange(tbil_grouped.shape[0]):
    addresses_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tbil_grouped.iloc[ind, :], num_top_venues)

addresses_venues_sorted

,Address,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"6 Grigol Robakidze Ave, Tbilisi, Georgia",Restaurant,Caucasian Restaurant,Hotel,Bakery,Café,Park,Outdoor Sculpture,Market,Shopping Mall,Bus Stop
1,"Tbilisi, A.Tsereteli ave.№95",Caucasian Restaurant,Bakery,Restaurant,Hotel,Park,Coffee Shop,Fast Food Restaurant,Nightclub,Café,Outdoor Sculpture
2,"Tbilisi, D.Guramishvili ave.№39b",Restaurant,Caucasian Restaurant,Shopping Mall,Supermarket,Park,Hotel,Furniture / Home Store,Lake,Market,Gym / Fitness Center
3,"Tbilisi, Gldani",Clothing Store,Fast Food Restaurant,Supermarket,Restaurant,Shopping Mall,Pool,Coffee Shop,Café,Gym / Fitness Center,Bakery
4,"Tbilisi, I.Chavchavadze av.№21",Hotel,Restaurant,Café,Plaza,Caucasian Restaurant,Theater,Hostel,Bar,Bakery,Park
5,"Tbilisi, Kakheti Highway №34b",Hotel,Caucasian Restaurant,Restaurant,Supermarket,Bed & Breakfast,Park,Gym Pool,Bakery,Shopping Mall,Coffee Shop
6,"Tbilisi, Ketevan Tsamebuli ave.№84/86",Café,Hotel,Wine Bar,Caucasian Restaurant,Theater,Restaurant,Hostel,Bar,Lounge,Park
7,"Tbilisi, Lilo Settlement, II kvart, building №9",Caucasian Restaurant,Bakery,Restaurant,Hotel,Park,Coffee Shop,Fast Food Restaurant,Nightclub,Café,Outdoor Sculpture
8,"Tbilisi, Moscow avenue №44",Hotel,Caucasian Restaurant,Shopping Mall,Restaurant,Supermarket,Bed & Breakfast,Bakery,Gym Pool,Pizza Place,Ice Cream Shop
9,"Tbilisi, Pekini ave.№39",Bakery,Café,Caucasian Restaurant,Hotel,Park,Ice Cream Shop,Restaurant,Coffee Shop,Fast Food Restaurant,Gym


In [20]:
addresses_venues_sorted.shape

(14, 11)

In [ ]:
addresses_venues_sorted.to_csv('C:\\Users\\admin\\Desktop\\Job\\Project_1\\Grigor\\common_venue.csv', encoding='utf-8', index=True)

In [62]:
from sklearn.cluster import KMeans

# 100,500,6
# my_data = my_data.drop(23)
kclusters = 5

tbil_grouped_clustering = tbil_grouped.drop('Address', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tbil_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 0, 1, 4, 2, 3, 2, 0, 3, 0])

In [63]:
tbil_merged = my_data

tbil_merged['Cluster Labels'] = kmeans.labels_ 

tbil_merged = tbil_merged.join(addresses_venues_sorted.set_index('Address'), on='Address')

tbil_merged

,Unnamed: 0,Address,PostCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Tbilisi, D.Guramishvili ave.№39b",178,41.776998,44.796015,1,Restaurant,Caucasian Restaurant,Shopping Mall,Supermarket,Park,Hotel,Furniture / Home Store,Lake,Market,Gym / Fitness Center
1,1,"Tbilisi, Ketevan Tsamebuli ave.№84/86",144,41.691145,44.816181,0,Café,Hotel,Wine Bar,Caucasian Restaurant,Theater,Restaurant,Hostel,Bar,Lounge,Park
2,2,"Tbilisi, Station Square №2",102,41.722450,44.795888,1,Hotel,Bakery,Caucasian Restaurant,Restaurant,Ice Cream Shop,Café,Park,Theater,Nightclub,Plaza
3,3,"6 Grigol Robakidze Ave, Tbilisi, Georgia",159,41.756270,44.774395,4,Restaurant,Caucasian Restaurant,Hotel,Bakery,Café,Park,Outdoor Sculpture,Market,Shopping Mall,Bus Stop
4,4,"Tbilisi, A.Tsereteli ave.№95",119,41.749378,44.777163,2,Caucasian Restaurant,Bakery,Restaurant,Hotel,Park,Coffee Shop,Fast Food Restaurant,Nightclub,Café,Outdoor Sculpture
5,5,"Tbilisi, Lilo Settlement, II kvart, building №9",198,41.749378,44.777163,3,Caucasian Restaurant,Bakery,Restaurant,Hotel,Park,Coffee Shop,Fast Food Restaurant,Nightclub,Café,Outdoor Sculpture
6,6,"Tbilisi, Moscow avenue №44",137,41.671794,44.885657,2,Hotel,Caucasian Restaurant,Shopping Mall,Restaurant,Supermarket,Bed & Breakfast,Bakery,Gym Pool,Pizza Place,Ice Cream Shop
7,7,"Tbilisi, Kakheti Highway №34b",190,41.685897,44.861623,0,Hotel,Caucasian Restaurant,Restaurant,Supermarket,Bed & Breakfast,Park,Gym Pool,Bakery,Shopping Mall,Coffee Shop
8,8,"Tbilisi, V.Gorgasali str.№4",114,41.765252,44.751852,3,Restaurant,Caucasian Restaurant,Hotel,Market,Outdoor Sculpture,Shopping Mall,Eastern European Restaurant,Café,Furniture / Home Store,Park
9,9,"Tbilisi, I.Chavchavadze av.№21",179,41.700457,44.794320,0,Hotel,Restaurant,Café,Plaza,Caucasian Restaurant,Theater,Hostel,Bar,Bakery,Park


In [64]:
tbil_merged.to_csv('C:\\Users\\admin\\Desktop\\Job\\Project_1\\Grigor\\tbil_merged.csv', encoding='utf-8', index=True)

In [65]:
print(item['name'])

NameError: name 'item' is not defined

In [66]:
v['venue']['name']

NameError: name 'v' is not defined